In [1]:
import numpy as np
import heapq
import matplotlib.pyplot as plt
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
import pickle

In [2]:
text = open('sequence-data.txt').read().lower()
print('length of the data is: :', len(text))

length of the corpus is: : 581888


In [3]:
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [4]:
char_indices

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '#': 4,
 '$': 5,
 '%': 6,
 '&': 7,
 "'": 8,
 '(': 9,
 ')': 10,
 '*': 11,
 ',': 12,
 '-': 13,
 '.': 14,
 '/': 15,
 '0': 16,
 '1': 17,
 '2': 18,
 '3': 19,
 '4': 20,
 '5': 21,
 '6': 22,
 '7': 23,
 '8': 24,
 '9': 25,
 ':': 26,
 ';': 27,
 '?': 28,
 '@': 29,
 '[': 30,
 ']': 31,
 '_': 32,
 'a': 33,
 'b': 34,
 'c': 35,
 'd': 36,
 'e': 37,
 'f': 38,
 'g': 39,
 'h': 40,
 'i': 41,
 'j': 42,
 'k': 43,
 'l': 44,
 'm': 45,
 'n': 46,
 'o': 47,
 'p': 48,
 'q': 49,
 'r': 50,
 's': 51,
 't': 52,
 'u': 53,
 'v': 54,
 'w': 55,
 'x': 56,
 'y': 57,
 'z': 58,
 '£': 59,
 '½': 60,
 'à': 61,
 'â': 62,
 'æ': 63,
 'è': 64,
 'é': 65,
 'œ': 66,
 '—': 67,
 '‘': 68,
 '’': 69,
 '“': 70,
 '”': 71,
 '\ufeff': 72}

In [5]:
print(f'unique chars: {len(chars)}')

unique chars: 73


In [6]:
SEQUENCE_LENGTH = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - SEQUENCE_LENGTH, step):
    sentences.append(text[i: i + SEQUENCE_LENGTH])
    next_chars.append(text[i + SEQUENCE_LENGTH])
print('num training examples:',len(sentences))

num training examples: 193950


In [7]:
X = np.zeros((len(sentences), SEQUENCE_LENGTH, len(chars)), 
             dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [8]:
model = Sequential()
model.add(LSTM(128, input_shape=(SEQUENCE_LENGTH, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

In [19]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, y, validation_split=0.05, 
                    batch_size=128, epochs=5, shuffle=True)

Epoch 1/5
1440/1440 [==============================] - 132s 92ms/step - loss: 1.6208 - accuracy: 0.5144 - val_loss: 2.0614 - val_accuracy: 0.4333
Epoch 2/5
1440/1440 [==============================] - 132s 92ms/step - loss: 1.5211 - accuracy: 0.5416 - val_loss: 2.0251 - val_accuracy: 0.4497
Epoch 3/5
1440/1440 [==============================] - 145s 100ms/step - loss: 1.4688 - accuracy: 0.5562 - val_loss: 2.0316 - val_accuracy: 0.4503
Epoch 4/5
1440/1440 [==============================] - 138s 96ms/step - loss: 1.4356 - accuracy: 0.5654 - val_loss: 2.0340 - val_accuracy: 0.4516
Epoch 5/5
1440/1440 [==============================] - 142s 99ms/step - loss: 1.4099 - accuracy: 0.5722 - val_loss: 2.0535 - val_accuracy: 0.4556


In [20]:
def prepare_input(text):
    x = np.zeros((1, SEQUENCE_LENGTH, len(chars)))
    for t, char in enumerate(text):
        x[0, t, char_indices[char]] = 1.
    return x

In [21]:
prepare_input("This is an example of input for our LSTM".lower())

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [22]:
def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return heapq.nlargest(top_n, range(len(preds)), preds.take)

In [23]:
def predict_completion(text):
    original_text = text
    generated = text
    completion = ''
    while True:
        x = prepare_input(text)
        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, top_n=1)[0]
        next_char = indices_char[next_index]
        text = text[1:] + next_char
        completion += next_char

        if len(original_text + completion) + 2 > len(original_text) and next_char == ' ':
            return completion

In [24]:
def predict_completions(text, n=3):
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [indices_char[idx] + predict_completion(text[1:] + indices_char[idx]) for idx in next_indices]


In [25]:
#seq=input("Enter Someting: ")
#print(predict_completions(seq.lower(), 5))

In [26]:
quotes = [
    "It is not a lack of love, but a lack of friendship that makes unhappy marriages.",
    "That which does not kill us makes us stronger.",
    "I'm not upset that you lied to me, I'm upset that from now on I can't believe you.",
    "And those who were seen dancing were thought to be insane by those who could not hear the music.",
    "It is hard enough to remember my opinions, without also remembering my reasons for them!"
]

In [27]:
for q in quotes:
    seq = q[:40].lower()
    print(seq)
    print(predict_completions(seq, 5))
    print()

it is not a lack of love, but a lack of 
['the ', 'me ', 'a ', 'his ', 'it ']

that which does not kill us makes us str
['eet ', 'ang ', 'ick ', 'ong ', 'uck ']

i'm not upset that you lied to me, i'm u
['pon ', 's ', 'nder ', 'ltical ', 'cced ']

and those who were seen dancing were tho
['ugh ', 'se ', 'te ', 're ', 'wer ']

it is hard enough to remember my opinion
[' of ', ', ', '. ', 's ', '\nof ']

